In [117]:
from datetime import datetime
from datetime import timezone
import polars as pl
import os

acropolis_directory = "/Users/patrickaigner/Documents/PROJECTS/ACROPOLIS/Software/quickflow/acropolis/"

In [118]:
df = pl.scan_parquet(os.path.join(acropolis_directory, "pivot_measurements.parquet"))

df = df.sort("creation_timestamp").filter(pl.col("system_name") == "tum-esm-midcost-raspi-2" ).sort(pl.col("creation_timestamp"))
df.tail(5).collect()

system_name,revision,creation_timestamp,receipt_timestamp,wxt532_direction_min,wxt532_speed_min,wxt532_last_update_time,wxt532_speed_avg,wxt532_direction_avg,wxt532_direction_max,wxt532_speed_max,gmp343_compensated,bme280_humidity,sht45_temperature,bme280_pressure,sht45_humidity,gmp343_raw,bme280_temperature,gmp343_filtered,gmp343_temperature,enclosure_bme280_humidity,raspi_cpu_usage,enclosure_bme280_pressure,raspi_cpu_temperature,raspi_disk_usage,raspi_memory_usage,enclosure_bme280_temperature,cal_gmp343_compensated,cal_gmp343_raw,cal_bme280_temperature,cal_gmp343_filtered,cal_sht45_temperature,cal_bme280_pressure,cal_sht45_humidity,cal_gmp343_temperature,cal_bme280_humidity,cal_bottle_id
str,i32,"datetime[ns, UTC]","datetime[ns, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""tum-esm-midcos…",18,2023-10-13 14:22:32.970 UTC,2023-10-13 14:22:34.505739 UTC,null,null,null,null,null,null,null,405.3,15.45,31.12,934.16,19.81,357.5,31.04,410.3,32.6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""tum-esm-midcos…",18,2023-10-13 14:22:42.980 UTC,2023-10-13 14:22:43.575448 UTC,null,null,null,null,null,null,null,404.9,15.33,31.1,935.39,19.63,356.3,31.04,404.1,32.6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""tum-esm-midcos…",18,2023-10-13 14:22:52.990 UTC,2023-10-13 14:22:55.701969 UTC,null,null,null,null,null,null,null,404.0,15.23,31.13,936.08,19.56,356.1,31.04,404.8,32.6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""tum-esm-midcos…",18,2023-10-13 14:23:03 UTC,2023-10-13 14:23:04.669123 UTC,null,null,null,null,null,null,null,402.9,15.17,31.12,934.92,19.48,355.4,31.04,402.0,32.6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""tum-esm-midcos…",18,2023-10-13 14:23:13.010 UTC,2023-10-13 14:23:13.706551 UTC,null,null,null,null,null,null,null,409.7,15.33,31.12,935.63,19.68,360.8,31.04,405.0,32.6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [119]:
df = df.with_columns(pl.when(pl.col("cal_gmp343_filtered") < 600).then(pl.col("cal_gmp343_filtered")).otherwise(None).alias("cal_400"))

df = df.with_columns(pl.when(pl.col("cal_gmp343_filtered") > 600).then(pl.col("cal_gmp343_filtered")).otherwise(None).alias("cal_800"))

In [120]:
dfg = df.groupby(pl.col("creation_timestamp").dt.day()).agg([pl.col("cal_400").drop_nulls(),
                                                             pl.col("cal_800").drop_nulls()]).collect()
dfg

creation_timestamp,cal_400,cal_800
u32,list[f64],list[f64]
24,[],[]
8,[],[]
9,[],[]
25,[],[]
1,[],[]
2,[],[]
26,[],[]
10,[],[]
27,[],[]


In [121]:
def average_bottle(data):
    data = data.to_list()
    #2nd bottle
    if 0 < len(data) < 70:
        x = data[int(len(data)*0.3):int(len(data)*0.95)]
        return sum(x) / len(x)
    #1st bottle
    elif len(data) > 70:
        x = data[int(len(data)*0.5):int(len(data)*0.95)]
        return sum(x) / len(x)
    else:
        return 0.0

In [122]:
dfg.select([pl.col("creation_timestamp"),
            pl.col("cal_400").apply(lambda x: average_bottle(x)).alias("mean_cal_400"),
            pl.col("cal_800").apply(lambda x: average_bottle(x)).alias("mean_cal_800")])

creation_timestamp,mean_cal_400,mean_cal_800
u32,f64,f64
24,0.0,0.0
8,0.0,0.0
9,0.0,0.0
25,0.0,0.0
1,0.0,0.0
2,0.0,0.0
26,0.0,0.0
10,0.0,0.0
27,0.0,0.0
